# Important Note:

To run this script you need to install the python library <b>*chardet*</b> (type *pip install chardet* in your terminal)

*chardet* helps to detect encodings of files.

This notebook is a condensed version of an exploratory notebook where a lot of code cells with test prints have been removed.

93 CSV files were downloaded from
https://www.data.gouv.fr/en/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/

(date of statement 27.03.2025)

After an initial exploration of the data files and reading the documentation, I decided to use:

The Etalab database on bodily injury road accidents for a given year is divided into four categories, each represented as a separate CSV file:

1. CARACTÉRISTIQUES – Describes the general circumstances of the accident.

2. LIEUX – Describes the main location of the accident, even if it occurred at an intersection.

3. VÉHICULES – Lists the vehicles involved in the accident.

4. USAGERS – Describes the road users involved in the accident.

The database seems to have the highest level of detail.


#  Goal of this notebook
The goal is to create vertical stacked Dataframes by merging csv-files based on the categories mentioned above.


Step 1: Loading All Relevant CSV Files and Converting Them into DataFrames

Here is a list of problems I encountered during initial test runs:

- Some CSV files are encoded differently 
  the package chardet is able to identify encodings, but may not work in all cases.

- Some CSV files have different separators (I tried pd.read_csv("file.csv", sep=None, engine="python"), but it didn’t work).

- Some CSV files have inconsistent file names.

- Some CSV files are not needed (at least for now).

The code below is "solving" the mentioned problems and creates DataFrames that are stored in a dictionary with the filename as the key for further analysis.

I implemented try and except blocks to handle exceptions in the code. What is this? These are errors that can occur during the execution of a program/cell. These errors can be things like division by zero, file not found, encoding problems, etc. 
Instead of letting the program crash when an error occurs, you use those errors to provoke iterations. This allows you to anticipate errors and define how the program should respond when they happen.

<b>One more important note: Make sure to modify folder_path to your environment.</b>

In [2]:
import os
import numpy as np
import pandas as pd
import chardet
from collections import Counter
import re


In [3]:
# Define the folder path where the CSV files are stored
folder_path = r"C:\DS_Project_RoadAcc\raw_data_2015-2023"

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Dictionary to store categorized data
dataframes = {
    "v": {},  # Vehicles
    "u": {},  # Usagers (Users)
    "l": {},  # Lieux (Locations)
    "c": {},  # Caracteristiques (Characteristics)
    "y": {},
    "not_chosen": {}  # Files that don’t match any category
}

# Categorize files directly in the dictionary
for file in csv_files:
    if "hicules" in file:
        category = "v"
    elif "usager" in file:
        category = "u"
    elif "lieu" in file:
        category = "l"
    elif "istiques" in file or "act" in file:
        category = "c"
    elif "donn" not in file and "9617e399bb6b" not in file and "07a3338614" not in file:
        category = "y"
    else:
        category = "not_chosen"

    # Store an empty dictionary for now (dataframes will be added later)
    dataframes[category][file] = None

# Print files that were not categorized
print("Files not chosen for now:", list(dataframes["not_chosen"].keys()))

# List of common encodings and separators
common_encodings = ['latin1', 'utf-8', 'utf-8-sig', 'utf-16', 'ISO-8859-1', 'ISO-8859-2', 'ascii']
common_separators = [',', ';', '\t', '|', ':']

# Load CSV files into the dictionary, **excluding "not_chosen" category**
for category_name, files in dataframes.items():
    if category_name == "not_chosen":  
        continue  # Skip "not_chosen" category

    for file in files.keys():
        file_path = os.path.join(folder_path, file)
        success = False  # Track whether the file was loaded successfully

        # Detect encoding with chardet
        with open(file_path, 'rb') as f:
            result = chardet.detect(f.read(1000))
        detected_encoding = result['encoding']

        # Try reading with detected encoding
        try:
            df = pd.read_csv(file_path, encoding=detected_encoding, sep=None, engine='python')
            dataframes[category_name][file] = df
            print(f"Successfully loaded {file} (Category: {category_name}) with encoding '{detected_encoding}'")
            success = True
        except Exception:
            pass  # Try alternative encodings and separators if this fails

        # Try different encodings and separators if needed
        if not success:
            for encoding in common_encodings:
                for separator in common_separators:
                    try:
                        df = pd.read_csv(file_path, encoding=encoding, sep=separator, engine='python')
                        dataframes[category_name][file] = df
                        print(f"Successfully loaded {file} (Category: {category_name}) with encoding '{encoding}' and separator '{separator}'")
                        success = True
                        break  # Stop trying if successful
                    except Exception:
                        continue  # Try next separator

                if success:
                    break  # Stop trying other encodings

        # If all methods fail, print an error message
        if not success:
            print(f"Unable to read {file} in category '{category_name}'.")

Files not chosen for now: []
Successfully loaded vehicules-2017.csv (Category: v) with encoding 'ascii'
Successfully loaded vehicules-2018.csv (Category: v) with encoding 'ascii'
Successfully loaded vehicules-2019.csv (Category: v) with encoding 'utf-8'
Successfully loaded vehicules-2020.csv (Category: v) with encoding 'utf-8'
Successfully loaded vehicules-2021.csv (Category: v) with encoding 'utf-8'
Successfully loaded vehicules-2022.csv (Category: v) with encoding 'utf-8'
Successfully loaded vehicules-2023.csv (Category: v) with encoding 'utf-8'
Successfully loaded vehicules_2015.csv (Category: v) with encoding 'ascii'
Successfully loaded vehicules_2016.csv (Category: v) with encoding 'ascii'
Successfully loaded usagers-2017.csv (Category: u) with encoding 'ascii'
Successfully loaded usagers-2018.csv (Category: u) with encoding 'ascii'
Successfully loaded usagers-2019.csv (Category: u) with encoding 'utf-8'
Successfully loaded usagers-2020.csv (Category: u) with encoding 'utf-8'
Succ

# Step 2: Vertical Stack of Files/DataFrames According to Their Category

In this step, I want to combine all dataframes that represent one category but are separated by year (e.g., vehicules_2011.csv, vehicules_2012.csv, vehicules_2013.csv, vehicules_2014.csv, etc.).

Here is a list of problems that need to be addressed to ensure consistency when stacking vertically:

- Ensure that column names are the same in every file (this includes whitespace and capitalization and columns with no names). 
  normalized_columns.replace('', 'unnamed') deals with empty column names 
  df.columns = df.columns.str.strip().str.lower() deals with whitespace and capitalization.

- Ensure that the columns have the same data type (int, str, float, etc.). This is not necessary when using pd.concat(). Columns with different types are upcasted automatically, and you can define the type later. Upcasting hierarchy: bool → int → float → str → object.
Not so true, better check for consistent data types, its a first good indicator if values are consistent within in the columns and dataframes you want to concat/merge

- if you know the columns should be integer or float but apearing to be object or string its worth it to investigate if there are special characters or letters in the values

- if they should be integer vut are float, they may have decimal numbers

- Ensure that the columns have the same order/alignment. This is not necessary when using pd.concat(), as alignment happens automatically.

- Check for duplicates after stacking.

- The number of columns should be equal in every dataframe. When using pd.concat(), columns that exist in dataframes will still be included, but missing values (NaN) will be filled in where data is absent.

<b>Summary: When using pd.concat(), the most important step to take when vertical stacking dataframes is ensuring that the corresponding column names in all frames are identical.</b>

The following code removes whitespace and capitalization and compares the count of dataframes in a category with the occurences of column names.

In [4]:
# Iterate through each category, excluding "not_chosen"
for category_name, category_files in dataframes.items():
    if category_name == "not_chosen" or category_name == "y":
        continue  # Skip files that were not categorized

    column_counter = Counter()  # Counter to track column occurrences

    # Process each file in the category
    for file, df in category_files.items():
        if df is not None:
            # Normalize the column names by stripping whitespace and converting to lowercase
            df.columns = df.columns.str.strip().str.lower()

            # Replace empty column names with a default name (e.g., 'unnamed')
            df.columns = [col if col != '' else 'unnamed' for col in df.columns]

            # Count occurrences of each normalized column name
            for column in df.columns:
                column_counter[column] += 1

    # Check for mismatches (where count != number of files)
    num_files = len(category_files)
    for column, count in column_counter.items():
        if count != num_files:
            print(f"Category: {category_name} Total count {num_files} | Column: '{column}' | count is: {count} <- mismatch")

    # Print separator after each category to make the output clearer
    print("-----")

Category: v Total count 9 | Column: 'id_vehicule' | count is: 5 <- mismatch
Category: v Total count 9 | Column: 'motor' | count is: 5 <- mismatch
-----
Category: u Total count 9 | Column: 'secu' | count is: 4 <- mismatch
Category: u Total count 9 | Column: 'id_vehicule' | count is: 5 <- mismatch
Category: u Total count 9 | Column: 'secu1' | count is: 5 <- mismatch
Category: u Total count 9 | Column: 'secu2' | count is: 5 <- mismatch
Category: u Total count 9 | Column: 'secu3' | count is: 5 <- mismatch
Category: u Total count 9 | Column: 'id_usager' | count is: 3 <- mismatch
-----
Category: l Total count 9 | Column: 'env1' | count is: 4 <- mismatch
Category: l Total count 9 | Column: 'vma' | count is: 5 <- mismatch
-----
Category: c Total count 9 | Column: 'num_acc' | count is: 8 <- mismatch
Category: c Total count 9 | Column: 'gps' | count is: 4 <- mismatch
Category: c Total count 9 | Column: 'accident_id' | count is: 1 <- mismatch
-----


## Interpretation of mismatches where the occurence of column names is not equal to the number of dataframes in a category

```Category: v | Column: 'id_vehicule' | Occurrences: 5 <- mismatch
Category: v | Column: 'motor' | Occurrences: 5 <- mismatch

Category: u | Column: 'secu' | Occurrences: 14 <- mismatch
Category: u | Column: 'id_vehicule' | Occurrences: 5 <- mismatch
Category: u | Column: 'secu1' | Occurrences: 5 <- mismatch
Category: u | Column: 'secu2' | Occurrences: 5 <- mismatch
Category: u | Column: 'secu3' | Occurrences: 5 <- mismatch
Category: u | Column: 'id_usager' | Occurrences: 3 <- mismatch

Category: l | Column: 'env1' | Occurrences: 14 <- mismatch
Category: l | Column: 'vma' | Occurrences: 5 <- mismatch

Category: c | Column: 'num_acc' | Occurrences: 18 <- mismatch
Category: c | Column: 'gps' | Occurrences: 14 <- mismatch
Category: c | Column: 'accident_id' | Occurrences: 1 <- mismatch
```

To gain more knowlege i read to the documentation of the dataset: "In 2019, the accident database was updated"
Regarding the columns names with the occurences of 5, the variables were introduced 2019 so 5 makes sence when having data up to the year 2023.
In reverse columns names with the occurences of 14 pointing towards variables which were excluded from 2019 on.


```
id_usager: Unique identifier for each user (including pedestrians linked to the vehicle that hit them) – Numeric code.
The variable seems to be introduced in 2021

In the following code the columns with wrong names get renamed:
- accident_id need to be renamed in num_acc
- 'type accident - libellé (old)' to 'type accident - libellé'


In [5]:
#Renaming accident_id to num_acc 
# Iterate through the files in categories "v", "u", "l", "c", "y"
for category_name in ["v", "u", "l", "c", "y"]:
    for file, df in dataframes[category_name].items():
        # Check if 'accident_id' column is present and rename it
        if 'accident_id' in df.columns or 'type accident - libellé (old)' in df.columns:
            df.rename(columns={
                'accident_id':'num_acc',
                'type accident - libellé (old)':'type accident - libellé'
                }, inplace=True)
            print(f"Renamed columns in file: {file} in category '{category_name}'.")

Renamed columns in file: carcteristiques-2022.csv in category 'c'.


## Check for consistent data types of columns in the same category

In [6]:

# Function to find mixed data type columns and display relevant details
def find_mixed_dtype_columns(category, df_dict):
    mixed_columns = {}

    # Aggregate data for each column across all DataFrames in the category
    for df in df_dict.values():
        for col in df.columns:
            if col not in mixed_columns:
                mixed_columns[col] = []
            mixed_columns[col].extend(df[col])

    # Process each column
    found_mixed = False
    for col, values in mixed_columns.items():
        unique_types = set(map(type, values))

        if len(unique_types) > 1:  # More than one unique data type
            found_mixed = True
            most_common_dtype = Counter(map(type, values)).most_common(1)[0][0]
            value_counts = pd.Series(values).value_counts()

            top_5 = value_counts.head(5).index.tolist()
            low_count_values = value_counts[value_counts == value_counts.min()]

            if len(low_count_values) > 5:
                low_count_values = low_count_values.sample(5)

            low_count_values = low_count_values.index.tolist()

            print(f"  Column '{col}' → Mixed types: {unique_types} | Most common: {most_common_dtype} | "
                  f"Top 5: {', '.join(map(str, top_5))} | "
                  f"Low count (random 5): {', '.join(map(str, low_count_values))}")

    if not found_mixed:
        print(f"\nNo mixed data types found in category: {category}")

# Iterate over each category, excluding "not_chosen" and "y"
for category, dfs in dataframes.items():
    if category == "not_chosen":
        continue
    print(f"\nChecking category: {category}")
    find_mixed_dtype_columns(category, dfs)


Checking category: v
  Column 'senc' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'int'> | Top 5: 1.0, 2.0, 0.0, 3.0, -1.0 | Low count (random 5): -1.0
  Column 'occutc' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'float'> | Top 5: 0.0, 1.0, 2.0, 3.0, 10.0 | Low count (random 5): 250.0, 78.0, 128.0, 210.0, 77.0
  Column 'obs' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'int'> | Top 5: 0.0, 1.0, 13.0, 2.0, 4.0 | Low count (random 5): -1.0
  Column 'obsm' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'int'> | Top 5: 2.0, 0.0, 1.0, 9.0, 6.0 | Low count (random 5): -1.0
  Column 'choc' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'int'> | Top 5: 1.0, 3.0, 2.0, 4.0, 8.0 | Low count (random 5): -1.0
  Column 'manv' → Mixed types: {<class 'int'>, <class 'float'>} | Most common: <class 'int'> | Top 5: 1.0, 2.0, 15.0, 0.0, 13.0 | Low count (random 5): -1.0

Checking cat

## Testing modifications of the upcast script

In [7]:
def find_and_upcast_mixed_dtype_columns(category, df_dict):
    mixed_columns = {}
    columns_with_strings = set()  # Track columns that contain strings

    # Aggregate data for each column across all DataFrames in the category
    for df in df_dict.values():
        for col in df.columns:
            if col not in mixed_columns:
                mixed_columns[col] = []
            mixed_columns[col].extend(df[col])  

    # Identify columns that contain strings
    for col, values in mixed_columns.items():
        unique_types = set(map(type, values))
        
        # If a column has any string, mark it for upcasting
        if str in unique_types or object in unique_types:
            columns_with_strings.add(col)

    # Convert columns as needed
    for col, values in mixed_columns.items():
        unique_types = set(map(type, values))

        # If this column is in the "string list", upcast all instances to string
        if col in columns_with_strings:
            for df in df_dict.values():
                if col in df.columns:
                    df[col] = df[col].astype(str)
                    print(f"Column '{col}' upcasted to string in all DataFrames")

        # If mixed int and float, convert to float64 (only if it wasn't marked for string)
        elif {int, float}.issubset(unique_types):
            for df in df_dict.values():
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype('float64')
                    print(f"Column '{col}' upcasted to float64")

# Iterate over each category, excluding "not_chosen"
for category, dfs in dataframes.items():
    if category == "not_chosen":
        continue
    print(f"\nUpcasting columns in category: {category}")
    find_and_upcast_mixed_dtype_columns(category, dfs)



Upcasting columns in category: v
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'senc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'occutc' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obs' upcasted to float64
Column 'obsm' upcasted to float64
Colum

## To do check if foat64 c variables can be downcasted to integer for better performance 
1. decimal parts who are 0 and in this case pass the column name to a dtype_conversion dictionary
2. if there is a decimal part, show me the most common uniques and the number with decimal part to decide if we keep float

## Rerunning mixed types test again

In [8]:
# Function to find mixed data type columns and display relevant details
def find_mixed_dtype_columns(category, df_dict):
    mixed_columns = {}

    # Aggregate data for each column across all DataFrames in the category
    for df in df_dict.values():
        for col in df.columns:
            if col not in mixed_columns:
                mixed_columns[col] = []
            mixed_columns[col].extend(df[col])

    # Process each column
    found_mixed = False
    for col, values in mixed_columns.items():
        unique_types = set(map(type, values))

        if len(unique_types) > 1:  # More than one unique data type
            found_mixed = True
            most_common_dtype = Counter(map(type, values)).most_common(1)[0][0]
            value_counts = pd.Series(values).value_counts()

            top_5 = value_counts.head(5).index.tolist()
            low_count_values = value_counts[value_counts == value_counts.min()]

            if len(low_count_values) > 5:
                low_count_values = low_count_values.sample(5)

            low_count_values = low_count_values.index.tolist()

            print(f"  Column '{col}' → Mixed types: {unique_types} | Most common: {most_common_dtype} | "
                  f"Top 5: {', '.join(map(str, top_5))} | "
                  f"Low count (random 5): {', '.join(map(str, low_count_values))}")

    if not found_mixed:
        print(f"\nNo mixed data types found in category: {category}")

# Iterate over each category, excluding "not_chosen" and "y"
for category, dfs in dataframes.items():
    if category == "not_chosen":
        continue
    print(f"\nChecking category: {category}")
    find_mixed_dtype_columns(category, dfs)


Checking category: v

No mixed data types found in category: v

Checking category: u

No mixed data types found in category: u

Checking category: l

No mixed data types found in category: l

Checking category: c

No mixed data types found in category: c

Checking category: y

No mixed data types found in category: y


## Time for the vertical merges in every category

In [9]:
# Iterate through each category, excluding "not_chosen"
for category_name in ["v", "u", "l", "c", "y"]:
    dataframes_to_merge = []  # List to hold DataFrames for each category
    merged_files = []  # List to store file names that were merged

    for file, df in dataframes[category_name].items():
        if df is not None:
            # Add the DataFrame to the list
            dataframes_to_merge.append(df)
            # Store the file name
            merged_files.append(file)

    # Check if there are DataFrames to merge
    if dataframes_to_merge:
        # Concatenate all DataFrames in the list vertically (along rows)
        merged_df = pd.concat(dataframes_to_merge, ignore_index=True)
        
        # Optionally, save the merged DataFrame to a new dictionary
        dataframes[category_name]["merged"] = merged_df
        
        # Print the shape of the merged DataFrame for the category
        print(f"Category: {category_name} - Merged DataFrame shape: {merged_df.shape}")
        print(f"Files merged for category '{category_name}': {', '.join(merged_files)}")

    # Print separator after each category to make the output clearer
    print("-----")


Category: v - Merged DataFrame shape: (871293, 11)
Files merged for category 'v': vehicules-2017.csv, vehicules-2018.csv, vehicules-2019.csv, vehicules-2020.csv, vehicules-2021.csv, vehicules-2022.csv, vehicules-2023.csv, vehicules_2015.csv, vehicules_2016.csv
-----
Category: u - Merged DataFrame shape: (1149961, 17)
Files merged for category 'u': usagers-2017.csv, usagers-2018.csv, usagers-2019.csv, usagers-2020.csv, usagers-2021.csv, usagers-2022.csv, usagers-2023.csv, usagers_2015.csv, usagers_2016.csv
-----
Category: l - Merged DataFrame shape: (525834, 19)
Files merged for category 'l': lieux-2017.csv, lieux-2018.csv, lieux-2019.csv, lieux-2020.csv, lieux-2021.csv, lieux-2022.csv, lieux-2023.csv, lieux_2015.csv, lieux_2016.csv
-----
Category: c - Merged DataFrame shape: (509796, 16)
Files merged for category 'c': caract-2023.csv, caracteristiques-2017.csv, caracteristiques-2018.csv, caracteristiques-2019.csv, caracteristiques-2020.csv, caracteristiques_2015.csv, caracteristiques_2

## Checking for duplicates

In [10]:
# Check for duplicates in each merged DataFrame
for category_name in ["v", "u", "l", "c", "y"]:
    if "merged" in dataframes[category_name]:  # Ensure merged data exists
        merged_df = dataframes[category_name]["merged"]

        # Find duplicate rows
        duplicates = merged_df[merged_df.duplicated()]
        
        if not duplicates.empty:
            print(f"Category: {category_name} - Found {len(duplicates)} duplicate rows:")
            print(duplicates)  # Print duplicate rows
        else:
            print(f"Category: {category_name} - No duplicates found.")
    else:
        print(f"Category: {category_name} - No merged DataFrame found.")



Category: v - No duplicates found.
Category: u - Found 796 duplicate rows:
              num_acc  place  catu  grav  sexe  trajet  secu  locp actp  \
1600     201700000700    5.0     2     1     1     5.0  13.0   0.0  0.0   
3387     201700001487    7.0     2     1     2     2.0  13.0   0.0  0.0   
3390     201700001487    7.0     2     3     2     2.0  11.0   0.0  0.0   
3393     201700001487    8.0     2     1     2     2.0  13.0   0.0  0.0   
3395     201700001487    8.0     2     1     2     2.0  13.0   0.0  0.0   
...               ...    ...   ...   ...   ...     ...   ...   ...  ...   
1145627  201600057549    5.0     2     4     1     0.0  11.0   0.0  0.0   
1145628  201600057549    5.0     2     4     2     0.0  11.0   0.0  0.0   
1145629  201600057549    5.0     2     4     1     0.0  11.0   0.0  0.0   
1146246  201600057814    2.0     2     4     1     0.0  12.0   0.0  0.0   
1149705  201600059294    NaN     3     4     1     5.0   NaN   1.0  3.0   

         etatp  an_nais 

In [11]:
#Iterate through each category and drop duplicates in the merged DataFrame
for category_name in ["v", "u", "l", "c", "y"]:
    if "merged" in dataframes[category_name]:  # Ensure merged data exists
        merged_df = dataframes[category_name]["merged"]
        
        # Drop duplicates and update the DataFrame
        cleaned_df = merged_df.drop_duplicates()

        # Store the cleaned DataFrame back in the dictionary
        dataframes[category_name]["merged"] = cleaned_df

        # Print the number of duplicates removed
        print(f"Category: {category_name} - {len(merged_df) - len(cleaned_df)} duplicate rows removed.")
        print(f"New shape after dropping duplicates: {cleaned_df.shape}")
    else:
        print(f"Category: {category_name} - No merged DataFrame found.")

Category: v - 0 duplicate rows removed.
New shape after dropping duplicates: (871293, 11)
Category: u - 796 duplicate rows removed.
New shape after dropping duplicates: (1149165, 17)
Category: l - 0 duplicate rows removed.
New shape after dropping duplicates: (525834, 19)
Category: c - 0 duplicate rows removed.
New shape after dropping duplicates: (509796, 16)
Category: y - No merged DataFrame found.


## Saving the vertical stacked dataframes as csv-files

In [12]:
import os

# Define the output folder path
output_folder = r"C:\DS_Project_RoadAcc\merged_data_2015-2023"

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through each category and save the merged DataFrame
for category_name in ["v", "u", "l", "c", "y"]:
    if "merged" in dataframes[category_name]:  # Ensure merged data exists
        merged_df = dataframes[category_name]["merged"]
        
        # Define the file path
        file_path = os.path.join(output_folder, f"{category_name}_2015-2023.csv")

        # Save the DataFrame as a CSV file
        merged_df.to_csv(file_path, index=False)

        print(f"Category: {category_name} - Merged DataFrame saved to {file_path}")
    else:
        print(f"Category: {category_name} - No merged DataFrame to save.")

Category: v - Merged DataFrame saved to C:\DS_Project_RoadAcc\merged_data_2015-2023\v_2015-2023.csv
Category: u - Merged DataFrame saved to C:\DS_Project_RoadAcc\merged_data_2015-2023\u_2015-2023.csv
Category: l - Merged DataFrame saved to C:\DS_Project_RoadAcc\merged_data_2015-2023\l_2015-2023.csv
Category: c - Merged DataFrame saved to C:\DS_Project_RoadAcc\merged_data_2015-2023\c_2015-2023.csv
Category: y - No merged DataFrame to save.
